In [26]:
import MySQLdb as mysql
import math
import os
from six.moves.urllib.request import urlretrieve
import collections
import zipfile
import csv
import numpy as np
import tensorflow as tf
import random
# 有待改进， 可以首先通 pr 算法生成相应的 关键词，然后再优化词向量的筛选过程，getWords 使词向量都是 同一个文章内的的关键字，然后去看效果


In [45]:
vocabulary_size = 20033;
csvPath = "/Users/tianyi/project/dl-workshop-rnn-lstm/track_key.csv"

def maybe_download(filename, expected_bytes):
    """Download a file if not present, and make sure it's the right size."""
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    if statinfo.st_size == expected_bytes:
        print('Found and verified %s' % filename)
    else:
        print(statinfo.st_size)
        raise Exception(
        'Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename

def read_data(filename):
    """Extract the first file enclosed in a zip file as a list of words"""
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data
  
def testGetWords():
    filename = maybe_download('text8.zip', 31344016)
    words = read_data(filename)
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
        unk_count = unk_count + 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
    return data, count, dictionary, reverse_dictionary
dataEncode, count, dictionary, reverse_dictionary = testGetWords() 
print("count is " , len(count))
print(count[:5])

Found and verified text8.zip
('count is ', 20033)
[['UNK', 17005207], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]


In [11]:
conn = mysql.connect('localhost', 'root', '', 'analyze')
def GetList(sql):
    cur = conn.cursor()
    cur.execute(sql)
    data = cur.fetchall()
    return data
dlist = GetList("select count(*) from track_key")
print(len(dlist))
print(dlist)

1
((9120030L,),)


In [47]:
def putWordsText():
     # 只要名次
    sql = """ select track_key.word, track_key.count, track_key.article_id, word_attr.flag  from track_key inner join word_attr on 
            word_attr.word = track_key.word where word_attr.flag in 
            (select distinct flag as flag  from word_attr where flag like '%n%')  and track_key.is_delete = 0
    """
    print(sql)
    keys = GetList(sql)
    fp = open(csvPath, 'a')
    header = ["word", "count", "article_id", "flag"]
    csv_writer = csv.writer(fp)
    csv_writer.writerow(header)
    for line in keys:
        csv_writer.writerow(line)
    fp.close()
    print "count : ", len(keys)
    print "save to %s" % csvPath
putWordsText() 

 select track_key.word, track_key.count, track_key.article_id, word_attr.flag  from track_key inner join word_attr on 
            word_attr.word = track_key.word where word_attr.flag in 
            (select distinct flag as flag  from word_attr where flag like '%n%')  and track_key.is_delete = 0
    
count :  4569576
save to /Users/tianyi/project/dl-workshop-rnn-lstm/track_key.csv


In [42]:
def getWordsText():
    with open(csvPath, "r") as f:
        reader = csv.DictReader(f)
        column = []
        for row in reader:
            column.extend([row])
            #print row["count"]
        #print column[0]["count"
    return 
getWordsText()

1


In [44]:
def getWords(): 
    # 只要名次
    sql = """ select track_key.*, word_attr.flag  from track_key inner join word_attr on 
            word_attr.word = track_key.word where word_attr.flag in 
            (select distinct flag as flag  from word_attr where flag like '%n%')
    """
    #print(sql)
    #keys = GetList(sql)
    words = {}
    keys = []
    with open(csvPath, "r") as f:
        reader = csv.DictReader(f)
        for row in reader:
            keys.extend([row])
    for key in keys:
        word = key["word"]
        artId = key["article_id"]
        cnt = key["count"]
        if not words.has_key(artId):
            words[artId] = {}
        words[artId][word] = cnt 

    allWords = []
    for art in words:
        allWords.extend(words[art].keys())
    count = [['UNK', -1]]
    count.extend(collections.Counter(allWords).most_common(vocabulary_size))

    cnt = 0
    dictionary = {}
    for word,_ in count:
        dictionary[word] = cnt
        cnt = cnt + 1
    dataEncode = list()
    unk_count = 0
    for word in allWords:
        index = 0
        if word in dictionary:
            index = dictionary[word]
        else:
            unk_count = unk_count + 1
        dataEncode.append(index)
    count[0][1] = unk_count

    reverse_dictinary = dict(zip(dictionary.values(), dictionary.keys()))
    return dataEncode, count, dictionary, reverse_dictinary
_, count, _, _ = getWords() 

print(count[:10])



[['UNK', 0], ('A\xe8\x82\xa1', 84), ('AT&T', 14), ('T\xe6\x81\xa4', 1), ('C++', 1)]


In [4]:
data_index = 0
def generate_batch(batch_size, num_skips, skip_window, data):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]        
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
    return batch, labels
#dataEncode, count, dictionary, reverse_dictionary = testGetWords() 
for num_skips, skip_window in [(2, 1), (4, 2)]:
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window, data=dataEncode)
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    for val in batch:
        print val, 
        print "_______________"
    for val in labels:
        print val, 
    #print " ".join(batch)
    #print " ".join(labels)

    #print('    batch:', [reverse_dictionary[bi] for bi in batch])
    #print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])


with num_skips = 2 and skip_window = 1:
3084 _______________
3084 _______________
3084 _______________
3084 _______________
3084 _______________
3084 _______________
3084 _______________
3084 _______________
[5239] [12] [5239] [12] [12] [5239] [5239] [12] 
with num_skips = 4 and skip_window = 2:
12 _______________
12 _______________
12 _______________
12 _______________
12 _______________
12 _______________
12 _______________
12 _______________
[5239] [6] [195] [3084] [5239] [3084] [195] [6]


In [23]:
print __name__
embedding_size = 128
stepsNum = 10001;
valid_window = 100
valid_size = 16
batch_size = 256
# 随机选择用来测试 运算效果的case
valid_examples = np.array(random.sample(range(valid_window), valid_size))
if __name__ == "__main__": 
    graph = tf.Graph()
    with graph.as_default(), tf.device('/cpu:0'):
        train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
        valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

        embedding = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        #biases = tf.zeros(batch_size) + 0.01
        biases = tf.Variable(tf.zeros([vocabulary_size]))

        softmax_weight = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size], stddev=1.0 / math.sqrt(embedding_size)))
        lookedEmbdding = tf.nn.embedding_lookup(embedding, train_dataset)
        num_sampled = 64
        loss = tf.reduce_mean(tf.nn.sampled_softmax_loss(
            weights=softmax_weight, 
            biases=biases,
            inputs=lookedEmbdding, 
            num_sampled=num_sampled,
            labels=train_labels, 
            num_classes=vocabulary_size)
        )
        optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)

        norm = tf.reduce_sum(tf.square(embedding), 1 , keep_dims=True)
        embd_norm = embedding / tf.sqrt(norm)
        valid_embeddings = tf.nn.embedding_lookup(embd_norm, valid_dataset)
        similarity = tf.matmul(embd_norm, tf.transpose(valid_embeddings))
    #dataEncode, count, dictionary, reverse_dictionary = testGetWords() 
    dataEncode, count, dictionary, reverse_dictionary = getWords() 

    print(len(dataEncode))

__main__
 select track_key.*, word_attr.flag  from track_key inner join word_attr on 
            word_attr.word = track_key.word where word_attr.flag in 
            (select distinct flag as flag  from word_attr where flag like '%n%')
    
4627469


In [24]:
#dataEncode, count, dictionary, reverse_dictionary = getWords() 
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    average_loss = 0
    calLossStep=1000
    print(stepsNum)
    for step in range(stepsNum):
        #batch_data, batch_labels = generate_batch(batch_size=batch_size, num_skips=2, skip_window=1, data=dataEncode)
        #batch_data, batch_labels = generate_batch(batch_size=batch_size, num_skips=2, skip_window=1, data=dataEncode)

        #print("data len is : ", len(batch_data), len(batch_labels))
        feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
        _, l = session.run([optimizer, loss], feed_dict=feed_dict)
        #print discard
        average_loss += l
        if (step % calLossStep) == 0:
            print average_loss / calLossStep
            average_loss = 0
        sim = similarity.eval()
        if (step % 1000) == 0:
            print "now get data %s" , step
            for i in range(valid_size):
                idx = valid_examples[i]
                valid_word = reverse_dictionary[idx]
                top_k = 8
                nearest = (-sim[i, :]).argsort()[1 : top_k + 1]
                log = 'Nearest to %s : ' % valid_word
                for top in range(top_k):
                    close_word = reverse_dictionary[nearest[top]]
                    log =  "%s,  %s\t" % (log, close_word)
                print log


Initialized
10001
0.0066512260437
now get data %s 0
Nearest to UNK : ,  投资	,  行业	,  市场	,  重点	,  板块	,  公司	,  中国	,  风险	
Nearest to 预期 : ,  公司	,  发展	,  方面	,  建议	,  企业	,  投资	,  板块	,  市场	
Nearest to 规模 : ,  UNK	,  重点	,  中国	,  板块	,  企业	,  风险	,  市场	,  方面	
Nearest to 建设 : ,  风险	,  行业	,  方面	,  投资	,  建议	,  UNK	,  发展	,  政策	
Nearest to 投资 : ,  方面	,  风险	,  预期	,  股份	,  市场	,  建议	,  行业	,  发展	
Nearest to 影响 : ,  政策	,  中国	,  预期	,  市场	,  UNK	,  板块	,  风险	,  股份	
Nearest to 数据 : ,  中国	,  公司	,  建议	,  发展	,  UNK	,  市场	,  风险	,  政策	
Nearest to 竞争 : ,  方面	,  板块	,  企业	,  风险	,  行业	,  市场	,  中国	,  重点	
Nearest to 估值 : ,  中国	,  建议	,  方面	,  企业	,  公司	,  投资	,  发展	,  风险	
Nearest to 环比 : ,  企业	,  发展	,  公司	,  行业	,  方面	,  股份	,  板块	,  建议	
Nearest to 国际 : ,  板块	,  企业	,  政策	,  中国	,  UNK	,  发展	,  投资	,  公司	
Nearest to 国家 : ,  发展	,  预期	,  UNK	,  企业	,  重点	,  股份	,  风险	,  公司	
Nearest to 公司 : ,  建议	,  预期	,  方面	,  投资	,  行业	,  发展	,  公司	,  板块	
Nearest to 销售 : ,  市场	,  重点	,  预期	,  建议	,  板块	,  政策	,  风险	,  公司	
Nearest to 布局 : ,  预期	,  市场	,  

0.0162901332632
now get data %s 8000
Nearest to UNK : ,  投资	,  行业	,  市场	,  重点	,  板块	,  公司	,  中国	,  风险	
Nearest to 预期 : ,  公司	,  发展	,  方面	,  建议	,  企业	,  投资	,  板块	,  市场	
Nearest to 规模 : ,  UNK	,  重点	,  中国	,  板块	,  企业	,  风险	,  市场	,  方面	
Nearest to 建设 : ,  风险	,  行业	,  方面	,  投资	,  建议	,  UNK	,  发展	,  政策	
Nearest to 投资 : ,  方面	,  风险	,  预期	,  股份	,  市场	,  建议	,  行业	,  发展	
Nearest to 影响 : ,  政策	,  中国	,  预期	,  市场	,  UNK	,  板块	,  风险	,  股份	
Nearest to 数据 : ,  中国	,  公司	,  建议	,  发展	,  UNK	,  市场	,  风险	,  政策	
Nearest to 竞争 : ,  方面	,  板块	,  企业	,  风险	,  行业	,  市场	,  中国	,  重点	
Nearest to 估值 : ,  中国	,  建议	,  方面	,  企业	,  公司	,  投资	,  发展	,  风险	
Nearest to 环比 : ,  企业	,  发展	,  公司	,  行业	,  方面	,  股份	,  板块	,  建议	
Nearest to 国际 : ,  板块	,  企业	,  政策	,  中国	,  UNK	,  发展	,  投资	,  公司	
Nearest to 国家 : ,  发展	,  预期	,  UNK	,  企业	,  重点	,  股份	,  风险	,  公司	
Nearest to 公司 : ,  建议	,  预期	,  方面	,  投资	,  行业	,  发展	,  公司	,  板块	
Nearest to 销售 : ,  市场	,  重点	,  预期	,  建议	,  板块	,  政策	,  风险	,  公司	
Nearest to 布局 : ,  预期	,  市场	,  建议	,  企业	,  风险	